# 💿 Dataset Conversion

This notebook converts raw robot recordings (`.mcap` files) into the LeRobot format required for training. 

The process involves:
1.  **Exploring** the available raw data.
2.  **Configuring** the dataset parameters (e.g., observations, actions).
3.  **Running** the conversion script.

--- 
## 1. Explore Raw Data

First, let's list the available raw data directories. Each directory contains a set of `.mcap` files from different teleoperation sessions.

In [11]:
!du -sh /data/raw/*

25G	/data/raw/0_practice
38M	/data/raw/12_disassemble-lego-set-4
13G	/data/raw/1_practice
131G	/data/raw/1_single-stack
13G	/data/raw/2_pick-place-red-block
254M	/data/raw/2_single-unstack
7.4G	/data/raw/3_sorting-color-based
291M	/data/raw/3_stack-two-bricks
17G	/data/raw/4_assembly-simple-stack
8.1G	/data/raw/5_lego-set-1
82G	/data/raw/5_sort-two-bricks
24M	/data/raw/6_unsort-two-bricks
18G	/data/raw/8_disassemble-lego-set-2


In [12]:
!du -sh /data/raw/5_sort-two-bricks/

82G	/data/raw/5_sort-two-bricks/


In [13]:
!du -sh /data/test/*

576K	/data/test/data
0	/data/test/images
76K	/data/test/meta
65M	/data/test/videos


In [14]:
# !rm -rf ~/data/test/

--- 
## 2. Configure Conversion

Now, specify the input and output paths and define the dataset's structure. 

> **Action Required:** Update `RAW_DATA_DIR` and `OUTPUT_DIR` below.

In [15]:
import pathlib
from example_policies.data_ops.config.pipeline_config import PipelineConfig, ActionMode

# --- Paths ---
# TODO: Set the input directory containing your .mcap files.
RAW_DATA_DIR = pathlib.Path("/data/raw/5_sort-two-bricks/anonymous")

# TODO: Set your desired output directory name.
OUTPUT_DIR = pathlib.Path("/data/sort_red_blocks_80")
# OUTPUT_DIR = pathlib.Path("/data/test")

# --- Configuration ---
# TODO: A descriptive label for the task, used for VLA-style text conditioning.
TASK_LABEL = "sort two red blocks"

cfg = PipelineConfig(
    task_name=TASK_LABEL,
    # Observation features to include in the dataset.
    include_tcp_poses=True,
    include_rgb_images=True,
    include_depth_images=False,
    # Action representation. TCP is a good default.
    action_level=ActionMode.TCP,
    # Subsampling and filtering. These are task-dependent.
    target_fps=10,
    max_pause_seconds=0.2,
    min_episode_seconds=1,
)

print(f"Input path:  {RAW_DATA_DIR}")
print(f"Output path: {OUTPUT_DIR}")

Input path:  /data/raw/5_sort-two-bricks/anonymous
Output path: /data/sort_red_blocks_50


--- 
## 3. Run Conversion

This cell executes the conversion process. It may take a while depending on the size of your data. You will see progress updates printed below.

In [16]:

from example_policies.data_ops.utils.conversion_utils import get_selected_episodes, get_sorted_episodes

# get_selected_episodes() only use success_good
# get_sorted_episodes() use all

episode_paths = get_selected_episodes(RAW_DATA_DIR, success_only=True)
# episode_paths = get_sorted_episodes(RAW_DATA_DIR)
episode_paths


[PosixPath('/data/raw/5_sort-two-bricks/anonymous/20251202_094748_task_success_good.mcap'),
 PosixPath('/data/raw/5_sort-two-bricks/anonymous/20251202_095420_task_success_good.mcap'),
 PosixPath('/data/raw/5_sort-two-bricks/anonymous/20251202_095528_task_success_good.mcap'),
 PosixPath('/data/raw/5_sort-two-bricks/anonymous/20251202_095652_task_success_good.mcap'),
 PosixPath('/data/raw/5_sort-two-bricks/anonymous/20251202_095834_task_success_good.mcap'),
 PosixPath('/data/raw/5_sort-two-bricks/anonymous/20251202_100251_task_success_good.mcap'),
 PosixPath('/data/raw/5_sort-two-bricks/anonymous/20251202_100723_task_success_good.mcap'),
 PosixPath('/data/raw/5_sort-two-bricks/anonymous/20251202_100911_task_success_good.mcap'),
 PosixPath('/data/raw/5_sort-two-bricks/anonymous/20251202_101133_task_success_good.mcap'),
 PosixPath('/data/raw/5_sort-two-bricks/anonymous/20251202_101353_task_success_good.mcap'),
 PosixPath('/data/raw/5_sort-two-bricks/anonymous/20251202_102313_task_success_g

In [17]:
# from datetime import datetime
# import re

# def filter_episodes_after(episode_paths, cutoff_dt):
#     """
#     Keep only episodes whose timestamp (parsed from filename) is after cutoff_dt.
    
#     filename example: 20251202_091423_prepare.mcap
#     """
#     filtered = []
#     pattern = re.compile(r"(\d{8}_\d{6})")   # match 20251202_091423
    
#     for path in episode_paths:
#         match = pattern.search(path.name)
#         if not match:
#             continue  # skip if no timestamp found
        
#         ts_str = match.group(1)
#         ts = datetime.strptime(ts_str, "%Y%m%d_%H%M%S")
        
#         if ts > cutoff_dt:
#             filtered.append(path)
    
#     return filtered


# # Example cutoff time
# cutoff = datetime(2025, 12, 2, 13, 40, 00)

# episode_paths = filter_episodes_after(episode_paths, cutoff)
episode_paths


[PosixPath('/data/raw/5_sort-two-bricks/anonymous/20251202_094748_task_success_good.mcap'),
 PosixPath('/data/raw/5_sort-two-bricks/anonymous/20251202_095420_task_success_good.mcap'),
 PosixPath('/data/raw/5_sort-two-bricks/anonymous/20251202_095528_task_success_good.mcap'),
 PosixPath('/data/raw/5_sort-two-bricks/anonymous/20251202_095652_task_success_good.mcap'),
 PosixPath('/data/raw/5_sort-two-bricks/anonymous/20251202_095834_task_success_good.mcap'),
 PosixPath('/data/raw/5_sort-two-bricks/anonymous/20251202_100251_task_success_good.mcap'),
 PosixPath('/data/raw/5_sort-two-bricks/anonymous/20251202_100723_task_success_good.mcap'),
 PosixPath('/data/raw/5_sort-two-bricks/anonymous/20251202_100911_task_success_good.mcap'),
 PosixPath('/data/raw/5_sort-two-bricks/anonymous/20251202_101133_task_success_good.mcap'),
 PosixPath('/data/raw/5_sort-two-bricks/anonymous/20251202_101353_task_success_good.mcap'),
 PosixPath('/data/raw/5_sort-two-bricks/anonymous/20251202_102313_task_success_g

In [18]:
from example_policies.data_ops.dataset_conversion import convert_episodes

convert_episodes(episode_paths, OUTPUT_DIR, cfg)

Processing /data/raw/5_sort-two-bricks/anonymous/20251202_094748_task_success_good.mcap...
  - Seen / Saved: 1678 / 300 in 0.02s | Total Time: 4.61s | FPS: 121.54
Saving /data/raw/5_sort-two-bricks/anonymous/20251202_094748_task_success_good.mcap processed with 1716 frames.


Map:   0%|          | 0/301 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Svt[info]: -------------------------------------------
Svt[info]: SVT [version]:	SVT-AV1 Encoder Lib v3.1.0
Svt[info]: SVT [build]  :	GCC 14.2.1 20250110 (Red Hat 14.2.1-7)	 64 bit
Svt[info]: LIB Build date: Oct  2 2025 21:17:50
Svt[info]: -------------------------------------------
Svt[info]: Level of Parallelism: 6
Svt[info]: Number of PPCS 305
Svt[info]: [asm level on system : up to avx512icl]
Svt[info]: [asm level selected : up to avx512icl]
Svt[info]: -------------------------------------------
Svt[info]: SVT [config]: main profile	tier (auto)	level (auto)
Svt[info]: SVT [config]: width / height / fps numerator / fps denominator 		: 256 / 256 / 10 / 1
Svt[info]: SVT [config]: bit-depth / color format 					: 8 / YUV420
Svt[info]: SVT [config]: preset / tune / pred struct 					: 8 / PSNR / random access
Svt[info]: SVT [config]: gop size / mini-gop size / key-frame type 			: 2 / 32 / key frame
Svt[info]: SVT [config]: BRC mode / rate factor 					: CRF / 30 
Svt[info]: SVT [config]: A

Saved episode 0 for main dataset.
Processing /data/raw/5_sort-two-bricks/anonymous/20251202_095420_task_success_good.mcap...
  - Seen / Saved: 862 / 174 in 0.02s | Total Time: 8.74s | FPS: 98.44
Saving /data/raw/5_sort-two-bricks/anonymous/20251202_095420_task_success_good.mcap processed with 870 frames.


...

Map:   0%|          | 0/175 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Svt[info]: -------------------------------------------
Svt[info]: SVT [version]:	SVT-AV1 Encoder Lib v3.1.0
Svt[info]: SVT [build]  :	GCC 14.2.1 20250110 (Red Hat 14.2.1-7)	 64 bit
Svt[info]: LIB Build date: Oct  2 2025 21:17:50
Svt[info]: -------------------------------------------
Svt[info]: Level of Parallelism: 6
Svt[info]: Number of PPCS 305
Svt[info]: [asm level on system : up to avx512icl]
Svt[info]: [asm level selected : up to avx512icl]
Svt[info]: -------------------------------------------
Svt[info]: SVT [config]: main profile	tier (auto)	level (auto)
Svt[info]: SVT [config]: width / height / fps numerator / fps denominator 		: 256 / 256 / 10 / 1
Svt[info]: SVT [config]: bit-depth / color format 					: 8 / YUV420
Svt[info]: SVT [config]: preset / tune / pred struct 					: 8 / PSNR / random access
Svt[info]: SVT [config]: gop size / mini-gop size / key-frame type 			: 2 / 32 / key frame
Svt[info]: SVT [config]: BRC mode / rate factor 					: CRF / 30 
Svt[info]: SVT [config]: A

Saved episode 51 for main dataset.

Metadata saved to /data/sort_red_blocks_50/meta

Processing LeRobot dataset post-conversion...
Processing episode file: episode_000000.parquet...
Processing episode file: episode_000001.parquet...
Processing episode file: episode_000002.parquet...
Processing episode file: episode_000003.parquet...
Processing episode file: episode_000004.parquet...
Processing episode file: episode_000005.parquet...
Processing episode file: episode_000006.parquet...
Processing episode file: episode_000007.parquet...
Processing episode file: episode_000008.parquet...
Processing episode file: episode_000009.parquet...
Processing episode file: episode_000010.parquet...
Processing episode file: episode_000011.parquet...
Processing episode file: episode_000012.parquet...
Processing episode file: episode_000013.parquet...
Processing episode file: episode_000014.parquet...
Processing episode file: episode_000015.parquet...
Processing episode file: episode_000016.parquet...
Pr

{'episode_mapping': {0: '/data/raw/5_sort-two-bricks/anonymous/20251202_094748_task_success_good.mcap',
  1: '/data/raw/5_sort-two-bricks/anonymous/20251202_095420_task_success_good.mcap',
  2: '/data/raw/5_sort-two-bricks/anonymous/20251202_095528_task_success_good.mcap',
  3: '/data/raw/5_sort-two-bricks/anonymous/20251202_095652_task_success_good.mcap',
  4: '/data/raw/5_sort-two-bricks/anonymous/20251202_095834_task_success_good.mcap',
  5: '/data/raw/5_sort-two-bricks/anonymous/20251202_100251_task_success_good.mcap',
  6: '/data/raw/5_sort-two-bricks/anonymous/20251202_100723_task_success_good.mcap',
  7: '/data/raw/5_sort-two-bricks/anonymous/20251202_100911_task_success_good.mcap',
  8: '/data/raw/5_sort-two-bricks/anonymous/20251202_101133_task_success_good.mcap',
  9: '/data/raw/5_sort-two-bricks/anonymous/20251202_101353_task_success_good.mcap',
  10: '/data/raw/5_sort-two-bricks/anonymous/20251202_102313_task_success_good.mcap',
  11: '/data/raw/5_sort-two-bricks/anonymous/

--- 
## ✅ Done!

Your new dataset is ready at the output path you specified. You can now proceed to the next notebook to train a policy.